# Unlock the Power of Machine Learning: Build a Content-Based Movie Recommendation System

## Import Libraries

In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from io import StringIO

## Load the Data

In [10]:
data = pd.read_csv('cleaned_animationmovies.csv')

## Data Preparation and Pre-processing

In [8]:
data['text_features'] = data['overview'].fillna('') + ' ' + data['tagline'].fillna('')

tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(data['text_features'].astype(str))

# Create a DataFrame with TF-IDF vectors
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Combine numerical features with TF-IDF vectors
combined_features = pd.concat([data[['vote_average', 'popularity', 'profit']], tfidf_df], axis=1)

# Calculate cosine similarity matrix
cosine_sim_matrix = cosine_similarity(combined_features, combined_features)

# Convert the cosine similarity matrix into a DataFrame for better readability
cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=data['title'], columns=data['title'])

## Build Recommendation Function

In [8]:
def get_recommendations(title):
    idx = data[data['title'] == title].index[0]
    sim_scores = list(enumerate(cosine_sim_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_movies = [(data['title'][i], i) for i, score in sim_scores[1:6]]
    return sim_movies

# Example: Get recommendations for a movie
movie_title = input("Enter a movie title: ")

while not movie_title.strip():
    print("Please enter a valid movie title.")
    movie_title = input("Enter a movie title: ")

recommendations = get_recommendations(movie_title)
print(f"Top 5 recommendations for '{movie_title}':")
for movie, idx in recommendations:
    print(f"- {movie}")

Enter a movie title:  Up


Top 5 recommendations for 'Up':
- How to Train Your Dragon: The Hidden World
- Inside Out
- WALL·E
- Ratatouille
- Brave
